In [11]:
import argparse
import os.path as osp
import glob

import numpy as np
import mmcv
import torch
import cv2
from mmedit.apis import init_model, restoration_inference
from mmedit.core import tensor2img
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image
import os

config = '../configs/restorers/glean/glean_ffhq_16x.py'
checkpoint = '../pretrained_model/glean_ffhq_16x_20210527-61a3afad.pth'
#video_path = '/data/jinsuyoo/datasets/MEAD/M009/video/front/happy/level_1/001.mp4'
video_path = '/data/jinsuyoo/datasets/MEAD/M009/frames/*/*/level_3/*/'
save_path = 'test_glean_mead'
device = 0

video_path = sorted(glob.glob(video_path))

model = init_model(config, checkpoint, device=torch.device('cuda', device))

for vp in tqdm(video_path):
    pid, _, angle, emo, lvl, filename = osp.normpath(vp).split(osp.sep)[-6:]

    os.makedirs(osp.join(save_path, pid, angle, emo, lvl, filename), exist_ok=True)

    lq_path = sorted(glob.glob(osp.join(vp, 'lq') + '/*.png'))
    gt_path = sorted(glob.glob(osp.join(vp, 'hq') + '/*.png'))

    #print(len(lq_path), len(gt_path))
  
    # save lq mp4
    lqs = []
    for img_path in lq_path:
        img = Image.open(img_path).convert('RGB')
        img = np.array(img)
        img = np.flip(img, axis=2)
        lqs.append(img)
    lqs = np.stack(lqs, axis=0)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(osp.join(save_path, pid, angle, emo, lvl, filename,'video_lq.mp4'), fourcc, 30, (64,64))
    for i in range(0, lqs.shape[0]):
        img = lqs[i]
        video_writer.write(img)
    video_writer.release()

    # save gt mp4
    gts = []
    for img_path in gt_path:
        img = Image.open(img_path).convert('RGB')
        img = np.array(img)
        img = np.flip(img, axis=2)
        gts.append(img)
    gts = np.stack(gts, axis=0)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(osp.join(save_path, pid, angle, emo, lvl, filename,'video_gt.mp4'), fourcc, 30, (1024, 1024))
    for i in range(0, gts.shape[0]):
        img = gts[i]
        video_writer.write(img)
    video_writer.release()

    results = []
    for i, img_path in enumerate(lq_path):
        output = restoration_inference(model, img_path)
        output = tensor2img(output)
        #mmcv.imwrite(output, osp.join(save_path, pid, angle, emo, lvl, filename, f'{i:03}_glean.png'))
        results.append(output)
    results = np.stack(results, axis=0)
    h, w= 1024, 1024 #results.shape[-2:]
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(osp.join(save_path, pid, angle, emo, lvl, filename,'video_glean.mp4'), fourcc, 30, (w, h))
    for i in range(0, results.shape[0]):
        img = results[i]
        video_writer.write(img)
    video_writer.release()



load checkpoint from http path: http://download.openmmlab.com/mmgen/stylegan2/official_weights/stylegan2-ffhq-config-f-official_20210327_171224-bce9310c.pth


2022-03-24 17:48:42,361 - mmedit - INFO - Load pretrained model from http://download.openmmlab.com/mmgen/stylegan2/official_weights/stylegan2-ffhq-config-f-official_20210327_171224-bce9310c.pth


load checkpoint from http path: http://download.openmmlab.com/mmgen/stylegan2/official_weights/stylegan2-ffhq-config-f-official_20210327_171224-bce9310c.pth


2022-03-24 17:48:44,883 - mmedit - INFO - Load pretrained model from http://download.openmmlab.com/mmgen/stylegan2/official_weights/stylegan2-ffhq-config-f-official_20210327_171224-bce9310c.pth
2022-03-24 17:48:46,785 - mmedit - INFO - load checkpoint from torchvision path: torchvision://vgg16


load checkpoint from local path: ../pretrained_model/glean_ffhq_16x_20210527-61a3afad.pth


...

 96%|█████████▌| 47/49 [25:51<01:05, 32.95s/it]

99 99


 98%|█████████▊| 48/49 [26:21<00:32, 32.14s/it]

123 123


100%|██████████| 49/49 [27:01<00:00, 33.08s/it]
